# Análise Geral dos dados

Este arquivo apresenta uma visão geral dos dados do RGPS no período de 1995 a 2016.

## Estrutura do banco de dados

Inicialmente é apresentada uma visão geral da estrutura do banco de dados.

In [1]:
import sqlite3
import pandas as pd
                        
# Função que executa as queries no banco de dados
def run_query(q):
    with sqlite3.connect('microdados/sqlite3/microdadosRGPS.db') as conn:
        return pd.read_sql(q, conn)

Lendo as tabelas do banco dados:

In [2]:
query = "SELECT name,type FROM sqlite_master WHERE type IN ('table')"
run_query(query)

,name,type
0,apos_aux,table
1,pensoes,table


O banco possui duas tabelas, uma com os dados das aposentadorias e auxílios (***apos_aux***) e outra com as pensões.

Em seguida, verificamos o esquema e quantidade de registros em cada uma das tabelas:

In [3]:
for tabela in ["apos_aux", "pensoes"]:
    registros = run_query('SELECT COUNT(ESPECIE) as count FROM {}'.format(tabela)).loc[0,'count']
    print("\nTabela: {} - {} registros".format(tabela,registros))
    print("Esquema da tabela:")
    print(run_query("pragma table_info(apos_aux);"))
    print("-"*40)    


Tabela: apos_aux - 57955872 registros
Esquema da tabela:
    cid           name     type  notnull dflt_value  pk
0     0        ESPECIE  INTEGER        0       None   0
1     1            DIB     TEXT        0       None   0
2     2            DDB     TEXT        0       None   0
3     3   MOT_CESSACAO  INTEGER        0       None   0
4     4  ULT_COMPET_MR  INTEGER        0       None   0
5     5          VL_MR     TEXT        0       None   0
6     6        DT_NASC     TEXT        0       None   0
7     7         VL_RMI     TEXT        0       None   0
8     8      CLIENTELA  INTEGER        0       None   0
9     9           SEXO  INTEGER        0       None   0
10   10       SITUACAO  INTEGER        0       None   0
11   11       DT_OBITO     TEXT        0       None   0
12   12      IDADE_DIB  INTEGER        0       None   0
13   13  TEMPO_CONTRIB  INTEGER        0       None   0
----------------------------------------

Tabela: pensoes - 7964185 registros
Esquema da tabela:
    c

Como observado a tabela ***apos_aux*** possui mais de 57 milhões de registros e a tabela ***pensoes*** possui quase 8 milhões de registros.
Além das quantidades de registros, a saída também apresenta os esquemas de cada umas das tabelas, sendo que ambas possuem 14 colunas.

A tabela abaixo apresenta uma breve descrição do significado das quatro primeiras colunas da tabela ***apos_aux***. 

|CAMPO|DESCRIÇÃO|
|---|---|
|ESPECIE|Espécie de Benefício|
|DIB|Data de Início do Benefício|
|DDB|Data de Despacho do Benefício|
|MOT_CESSACAO|Motivo de Cessação|

O arquivo ***metadados.xls***, disponibilizado junto com o banco de dados, apresenta uma descrição detalhada de ambas as tabelas.

A maioria dos dados é apresentada com códigos, assim, para o entendimento dos dados é necessário utilizar o arquivo ***dicionario.txt***, disponibilizado junto com banco, para que se possa entender os dados de cada coluna.
Abaixo segue uma parte do arquivo de dicionário:

* ESPÉCIE DE BENEFÍCIO (ESPECIE)

|CÓDIGO|VALOR|
|---|---|
|'01'|'B01 Pensão por morte do trab. rural'|    
|'02'|'B02 Pensão por morte acid. do trab. rural'|
|...||
|'95'|'B95 Auxílio-suplementar por acidente do trabalho'|
|'96'|'B96 Pensão especias às pessoas atingidas pela hanseníase'|


* VÍNCULO DE DEPENDENTE (VINC_DEP)

|CÓDIGO|VALOR|
|---|---|
|1|"Cônjuge"|
|2|"Filho(a)"| 
|...||
|9|"Dependente judicial"|
|99|"Ignorado"|
    

## Visão geral dos dados de Aposentadorias e Auxílios

Como primeira parte da análise, segue uma visão dos dados contidos na tabela ***apos_aux***.

Uma das principais colunas da tabela é a coluna *ESPECIE*, que contém o tipo de benefício. Então, vamos verificar quantos registros existem para cada tipo de benefício da tabela.

In [4]:
# O arquivo "dicionario_dados.py" possui os mesmos dados do "dicionario.txt", porém convertidos para dicionários Python
# Carrega as variáveis do arquivo de dicionário
%run util/dicionario_dados.py

# Executa a query e salva o DataFrame 
registros_por_benef = run_query("SELECT ESPECIE as Codigo, COUNT(ESPECIE) as qtd_registros FROM apos_aux \
          GROUP BY ESPECIE ORDER BY qtd_registros")

# Cria uma nova coluna com o nome do benefício
registros_por_benef['Beneficio'] = registros_por_benef['Codigo'].apply(lambda x: codigos_especie[x])
# Reordena as colunas
registros_por_benef = registros_por_benef[['Beneficio', 'Codigo', 'qtd_registros']]
registros_por_benef

,Beneficio,Codigo,qtd_registros
0,Aposentadoria por tempo de serviço de professor,57,114406
1,Aposentadoria por tempo de serviço especial,46,188462
2,Aposentadoria por invalidez por acidente do tr...,92,189969
3,Aposentadoria por invalidez previdenciária,32,3725842
4,Auxílio-doença por acidente do trabalho,91,4877036
5,Aposentadoria por tempo de serviço previdenciária,42,5272829
6,Aposentadoria por idade,41,10568276
7,Auxílio-doença previdenciário,31,33019052


Conforme a tabela acima, o benefício com código 31, que corresponde ao benefício *Auxílio-doença previdenciário* é o que possui mais registros, seguido da *Aposentadoria por idade*.

Foi observado que nem todos os benefícios estão contidos nos dados. 
O arquivo de dicionário descreve **78 tipos** de benefícios, sendo que 24 destes são do tipo *Pensão*, restando 54 tipos de benefícios descritos abaixo:

In [5]:
sorted([x for x in list(codigos_especie.values()) if 'Pens' not in x])

['Abono de permanência em serviço 20%',
 'Abono de permanência em serviço 25%',
 'Abono de servidor aposentado pela autarquia empreg.',
 'Amparo Assistencial ao idoso',
 'Amparo assistencial ao portador de deficiência',
 'Aposentadoria da extinta CAPIN',
 'Aposentadoria de extranumerário da União',
 'Aposentadoria excepcional do anistiado',
 'Aposentadoria por idade',
 'Aposentadoria por idade (Extinto Plano Básico)',
 'Aposentadoria por idade compulsória (Ex-SASSE)',
 'Aposentadoria por idade de ex-combatente marítimo',
 'Aposentadoria por idade do emp. rural',
 'Aposentadoria por idade do trab. rural',
 'Aposentadoria por invalidez (Ex-SASSE)',
 'Aposentadoria por invalidez (Extinto Plano Básico)',
 'Aposentadoria por invalidez acid. do trab. rural',
 'Aposentadoria por invalidez de aeronauta',
 'Aposentadoria por invalidez de ex-combatente marítimo',
 'Aposentadoria por invalidez do emp. rural',
 'Aposentadoria por invalidez do trab. rural',
 'Aposentadoria por invalidez por acident

De todos os benefícios listados acima somente **8** aparecem nos dados.

A ausência de dados de tantos benefícios (ex: salário maternidade, auxílio-reclusão, etc.) limita consideravelmente as análises que podem ser realizadas.

## Visão geral dos dados de Pensões

Semelhante ao realizado para a tabela anterior (apos_aux), abaixo segue uma visão geral dos dados da tabela com os dados de pensões

In [6]:
# Executa a query e salva o DataFrame 
registros_por_benef = run_query("SELECT ESPECIE as Codigo, COUNT(ESPECIE) as Registros FROM pensoes \
                                GROUP BY ESPECIE ORDER BY Registros")

# Cria uma nova coluna com o nome do benefício
registros_por_benef['Beneficio'] = registros_por_benef['Codigo'].apply(lambda x: codigos_especie[x])
# Reordena as colunas
registros_por_benef = registros_por_benef[['Beneficio', 'Codigo', 'Registros']]
registros_por_benef

,Beneficio,Codigo,Registros
0,Pensão por morte por acidente do trabalho,93,48675
1,Pensão por morte previdenciária,21,7915510


No caso das pensões, dos **24 tipos de pensões** descritas no dicionário de dados, somente **2** então presentes no banco de dados.

In [7]:
# analisar dados ausentes ou duplicados, etc.
# registros por ano

------------------
<span style="font-size: 12px"><i>
Em desenvolvimento...<br>
</i></span>